### DA 526 - Image Processing with Machine Learning
## Document Classification

### Instructor - Dr. Debanga Raj Neog


## Author_1 - Atul Bhagat
#### Roll Number - 224156004
#### Email - b.atul@iitg.ac.in
#### Date - 7th March 2023


We'll start with importing all the required libraries for this project.

We are using python-v8 on MacOS.

Required Libraries -
    Tensorflow
    Numpy
    Matplotlib
    Pandas

Note: It is important to install these libraries beforehand in your local environment. Creating a virtual environment is also highly recommended as it won't mess with the existing python (Global) libraries. Check the compatible tensorflow version and accordingly compatible numpy library must be installed. This information is available on their website.


In [ ]:
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, ZeroPadding2D
from tensorflow.keras import layers

print("TensorFlow_Version_", tf.__version__)

#### The first segment of the code initializes the necessary libraries and modules required for the project. It imports the following libraries and modules:

os: This library provides functions for interacting with the operating system, such as accessing file directories and performing system-related operations.

tensorflow (tf): TensorFlow is an open-source machine learning framework that offers a comprehensive set of tools and resources for building and deploying machine learning models.

matplotlib.pyplot: Matplotlib is a popular plotting library used for creating visualizations, including graphs and charts.

tensorflow.keras.models.Sequential: This module allows the construction of sequential models in TensorFlow, which are comprised of a linear stack of layers.

tensorflow.keras.layers: This module provides various types of layers used in neural network architectures, such as Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, and ZeroPadding2D.

tensorflow.keras: The keras module within TensorFlow offers a high-level API for designing and training neural networks.

In [ ]:
# import splitfolders
#
# input_folder = 'Data_Large/train'
# output_folder = 'Split'
#
# splitfolders.ratio(input=input_folder, output=output_folder,
#                    seed=1337, ratio=(.7, .15, .15), group_prefix=None, move=False)

#### Printing few basic environment details!

This code involves configuring GPU settings and printing system information. It performs the following tasks:

Checks if GPUs are available and sets memory growth to avoid memory errors.

Prints the environment variables for further analysis.

Prints the number of available GPUs.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth to True to avoid memory errors
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

for k, v in os.environ.items():
    print(f'{k} = {v}')

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# labels = pd.read_csv("train_labels.csv")

#### This code defines various configuration settings for the project. These settings include:

Image Size: The desired size (in pixels) for the input images. In this case, the variable i_s is set to 224.

Batch Size: The number of samples per gradient update during training. The variable batch_size is set to 32.

Epochs: The number of times the entire dataset will be passed through the model during training. The variable epochs is set to 30.

Patience: The number of epochs to wait before early stopping if no improvement in the validation metric is observed. The variable patience is set to 6.

Learning Rate: The rate at which the model's weights are updated during training. The variable learning_rate is set to 0.0001.

Optimizer: The optimization algorithm used for updating the model's weights. In this case, the variable opt is set to use the Adam optimizer with the specified learning rate.

Metrics: The evaluation metrics used to assess the model's performance. The variable metric is a list that includes accuracy, precision, and recall.

Loss Function: The loss function used to measure the model's performance during training. The variable loss is set to use categorical cross-entropy.

Data Augmentation: Various image augmentation techniques to enhance the diversity of the training data. These techniques include rotation, width and height shifts, shearing, zooming, horizontal flipping, and fill mode for pixel interpolation.

In [ ]:
#Image Size
i_s = 256

batch_size = 64
epochs = 30
patience = 6
learning_rate = 0.001
opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)

metric = ['accuracy']
loss = tf.losses.categorical_crossentropy

rotation_range = 10
width_shift_range = 0.4
height_shift_range = 0.4
shear_range = 0.3
zoom_range = 0.4
horizontal_flip = True
fill_mode = "nearest"

This snippet defines a function called extract_hidden_layer_data. This function is responsible for extracting and visualizing the activations of the hidden layers in a given model. Here's an overview of what the code does:

The function takes three parameters: path (the path to the saved model), sample (a sample image for visualization), and nc (the number of columns for the subplot grid).

The saved model is loaded using tf.keras.models.load_model.

A folder named "plots" is created if it doesn't exist, to save the generated plots.

For each convolutional layer (Conv2D) in the model, the code performs the following steps:

1. Prints the name of the current layer.
2. Sets up an activation model to obtain the layer outputs given the input image.
3. Predicts the activations for the sample image using the activation model.
4. Plots the activations for each filter in a grid.
5. Saves the plot as an image in the "plots" folder, organized by the model name and the layer name.

In [ ]:
def extract_hidden_layer_data(path, sample, nc):
    model = tf.keras.models.load_model(path)
    import os
    from matplotlib import pyplot as plt
    # Create a folder for saving the plots if it doesn't exist
    if not os.path.exists('plots'):
        os.makedirs('plots')

    # Getting a sample image
    x_test = sample
    # plt.imshow(x_test, cmap='Greys')
    # plt.title("Sample Image")

    for l_name in model.layers:
        if isinstance(l_name, tf.keras.layers.Conv2D):
            print("Layer -", l_name.name)
            layer_name = l_name.name
            layer_outputs = [layer.output for layer in model.layers if layer.name == layer_name]
            activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
            activations = activation_model.predict(x_test)

            # Plot the output

            # The i*4+j index is used to access the i-th row and j-th column of the array,
            # and then the imshow() method of the corresponding subplot is called to display the image.
            num_col = nc
            fig, ax = plt.subplots(nrows=l_name.filters // num_col, ncols=num_col, figsize=(60,60))
            for i in range(l_name.filters // num_col):
                for j in range(num_col):
                    ax[i, j].imshow(activations[0, :, :, i * num_col + j], cmap='gray')
                    ax[i, j].axis('off')

            # m_name = path.split("Saved_Models/")[1].split("_")[0]
            # print()
            # print("Current working directory:", os.getcwd())
            # export = f'plots/{m_name}/{l_name.name}'
            # print("Expected path:", export)
            # print()
            #
            # if not os.path.exists(export):
            #     os.makedirs(export)
            #
            # plt.savefig(f'{export}/{l_name.name}.png', bbox_inches='tight')
            plt.show()
            plt.close()

In [ ]:
def get_sample_image(i_s):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)
    sample_image = datagen.flow_from_directory('./Data/testlabeled/', target_size=(i_s, i_s),
                                               color_mode="grayscale",
                                               class_mode='categorical',
                                               subset='training',
                                               batch_size=1,shuffle='True')

    return sample_image.next()

In [ ]:
def print_samples(data):
    from matplotlib.gridspec import GridSpec

    img, label = data.next()
    print("BatchSize, TargetSize_H, TargetSize_W, Channels", img.shape)  #  (16,256,256,1)
    plt.rcParams["figure.autolayout"] = True

    fig = plt.figure(figsize=(10, 10))
    gs = GridSpec(3, 3, figure=fig)

    fig.suptitle(" == Sample Images == ")
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[0, 2])
    ax4 = fig.add_subplot(gs[1, 0])
    ax5 = fig.add_subplot(gs[1, 1])
    ax6 = fig.add_subplot(gs[1, 2])

    ax1.imshow(img[0], cmap="gray")
    ax2.imshow(img[1], cmap="gray")
    ax3.imshow(img[2], cmap="gray")
    ax4.imshow(img[3], cmap="gray")
    ax5.imshow(img[4], cmap="gray")
    ax6.imshow(img[5], cmap="gray")

    plt.show()
    plt.close()

## Data Loading and Augmentation

The fifth snippet of the code defines a function called get_data that handles the loading and augmentation of the dataset. Here's a breakdown of what the code does:

The function takes several parameters, including i_s (image size), batch_size, and various augmentation parameters such as rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip, fill_mode, and color.

Two instances of tf.keras.preprocessing.image.ImageDataGenerator are created: train_datagen and test_datagen. The train_datagen is used for data augmentation during training, while test_datagen is used for rescaling without augmentation during validation and testing.

The training data is loaded using train_datagen.flow_from_directory, which reads the images from the specified directory and applies the specified data augmentation techniques.

The validation and testing data are loaded using test_datagen.flow_from_directory. Unlike the training data, the validation and testing data are not augmented.

The function returns three iterators: train (for training data), val (for validation data), and test (for testing data).

In [ ]:
def get_data(i_s, batch_size, rotation_range=0, width_shift_range=0, height_shift_range=0, shear_range=0, zoom_range=0,
             horizontal_flip=False, fill_mode="nearest", color="grayscale"):
    # Here we are using inbuilt ImageDataGenerator from tensorflow package.
    # This caters to two of our needs - 1). This provides in-pipe data augmentation, that is, every image fed through this datagen would have some random augmentation done to it, to reduce over-fitting and this helps in generalising better.
    # 2) This works by creating a memory buffer, where we give it the path to our images, inside that path all the folder names would be considered as labels and each folder would contain images corresponding to that particular class.
    # Note : We've organised the data before-hand in this manner, so that we could use the flow_from_directory function directly.


    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.,
                                                                    rotation_range=rotation_range,
                                                                    width_shift_range=width_shift_range,
                                                                    height_shift_range=height_shift_range,
                                                                    shear_range=shear_range,
                                                                    zoom_range=zoom_range,
                                                                    horizontal_flip=horizontal_flip,
                                                                    fill_mode=fill_mode
                                                                    )

    # Note that the validation data should not be augmented!
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)


    # Since we have 16 classes in our dataset, we are labelling them 0-15 accordingly.
#     clas = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15']

    # A DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing a batch of images with shape (batch_size, *target_size, channels) and y is a numpy array of corresponding labels.
    train = train_datagen.flow_from_directory('./Data_Large/train', batch_size=batch_size, target_size=(i_s, i_s),
                                              color_mode=color,
                                              class_mode='categorical',
                                              shuffle=True)

    val = test_datagen.flow_from_directory('./Data_Large/val', target_size=(i_s, i_s),
                                           color_mode=color,
                                           class_mode='categorical'
                                           )

    test = test_datagen.flow_from_directory('./Data_Large/test', target_size=(i_s, i_s),
                                            color_mode=color,
                                            class_mode='categorical',
                                            shuffle=False)

    return train, val, test

In [ ]:
def start_training(model, model_name, epochs, train, val, factor=0.2, patience_reducelr=3, patience=5, min_lr=0.00001, training_samples='NoDataGiven'):
    import os
    from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


    monitor = "val_loss"
    enable_logging = 1

    log_dir = f"logs/fit/{model_name}___{training_samples}___samples"
    os.makedirs(log_dir, exist_ok=True)

    model_name_prefix = model_name.split("/")[0]
    model_filepath = f"Saved_Models/{model_name_prefix}/{model_name_prefix}_trained_on_{training_samples}_Data.h5"

    print(f"Model would be saved at {model_filepath}")

    reduce_lr = ReduceLROnPlateau(monitor=monitor,
                                  factor=factor,
                                  patience=patience_reducelr,
                                  min_lr=min_lr,
                                  cooldown = 1,
                                  verbose = enable_logging)

    early_stop = tf.keras.callbacks.EarlyStopping(monitor = monitor,
                                                  patience=patience,
                                                  start_from_epoch=3,
                                                  verbose = enable_logging
                                                 )
    tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                        histogram_freq=1,
                                        write_images=True
                                        )

    model_cp = ModelCheckpoint(filepath=model_filepath,
                               monitor=monitor,
                               save_best_only=True,
                               verbose=enable_logging)

    cb = [tb, reduce_lr, early_stop, model_cp]

    trained_model = model.fit(train,
                              epochs=epochs,
                              validation_data=val,
                              callbacks=cb,
                              verbose=enable_logging)
    print('**10')
    print("Model Training Ended")
    print(f"Saved Model at {model_filepath}")
    print('**10')
    print("(HISTORY) Parameters used for Model Fitting ")
    print(trained_model.params)

    return trained_model, model_filepath


## Model Training

The sixth snippet of the code defines a function called start_training that handles the training process for a given model. Here's a breakdown of what the code does:

The function takes several parameters, including the model to be trained, model_name, epochs, train data iterator, val data iterator, and various training-related parameters such as factor, patience_reducelr, patience, min_lr, and training_samples.

Several callbacks are set up for monitoring and saving the best model during training. These include ReduceLROnPlateau (to reduce learning rate on plateau), EarlyStopping (to stop training early if no improvement is observed), TensorBoard (to log training progress), and ModelCheckpoint (to save the best model during training).

The function creates a log directory to store the training logs and a directory to save the trained model.

The model is trained using the fit function, which takes the training and validation data iterators, the number of epochs, and the defined callbacks.

After training, the function prints information about the saved model and the parameters used for model fitting.

The function returns the trained model and the file path where the model is saved.

In [ ]:
import tensorflow as tf

def retrain_model(model_path, new_learning_rate, epochs, train, val, patience=3):
    try:
        loaded_model = tf.keras.models.load_model(model_path)
    except:
        print("Unable to load the model")

    learning_rate = new_learning_rate
    previous_opt = loaded_model.optimizer
    previous_opt.learning_rate.assign(learning_rate)

    # mn = loaded_model.__name__
    model_name = f'Retrained_'
    trained_model, model_filepath = start_training(loaded_model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))

    # Add best save callback
    save_callback = tf.keras.callbacks.ModelCheckpoint(filepath='Retrained_Model_{model_name}.h5',
                                                       monitor='val_loss',
                                                       save_best_only=True)
    trained_model.fit(train,
                      validation_data=val,
                      epochs=epochs,
                      callbacks=[save_callback])

    return trained_model, model_filepath


## Retraining the Model

This code snippet defines a function called retrain_model that allows retraining a pre-trained model with a new learning rate. Here's a breakdown of what the code does:

The function takes several parameters, including the model_path (path to the pre-trained model), new_learning_rate, epochs, train data iterator, val data iterator, and patience.

The pre-trained model is loaded using tf.keras.models.load_model function and stored in the loaded_model variable.

The learning rate of the optimizer in the loaded model is updated with the new learning rate.

The function generates a new model name for the retrained model and calls the start_training function with the loaded model, new model name, and other parameters for training.

The start_training function is responsible for training the model with the provided data and parameters.

Finally, the function returns the trained model and the file path where the model is saved.

In [ ]:
def evaluate_saved_model(model_filepath, test):
    import tensorflow as tf
    import numpy as np

    model = tf.keras.models.load_model(model_filepath)
    predictions = model.predict(test)

    y_true = test.labels
    y_pred_labels = np.argmax(predictions, axis=-1)

    # Compute the confusion matrix
    conf_mat = tf.math.confusion_matrix(y_true, y_pred_labels)

    print("Confusion Matrix:")
    print(conf_mat)

    return conf_mat


## Confusion Matrix and Classification Report

The eighth code snippet defines a function called confusion_matrix that calculates and visualizes the confusion matrix and classification report for a given model and test dataset. Here's a breakdown of what the code does:

The function takes the model_filepath (path to the trained model file) and test data iterator as input.

The trained model is loaded using tf.keras.models.load_model function.

Predictions are obtained using the loaded model on the test dataset.

The true labels (y_true) are extracted from the test dataset.

The function iterates through the predictions and compares them with the true labels to identify misclassifications. It collects misclassification information in the error_list and calculates the accuracy.

The function calculates the F1-score using sklearn.metrics.f1_score and prints it.

The confusion matrix is computed using sklearn.metrics.confusion_matrix and printed.

The classification report is generated using sklearn.metrics.classification_report and printed.

If the number of classes is less than or equal to 16, a heatmap of the confusion matrix is plotted using seaborn.heatmap and saved as a JPEG file.

Finally, the confusion matrix and classification report are returned.

In [ ]:
 def confusion_matrix(model_filepath, test):
    import tensorflow as tf
    import numpy as np

    model = tf.keras.models.load_model(model_filepath)
    predictions = model.predict(test)

    y_true = test.labels
    # y_pred_labels = np.argmax(predictions, axis=-1)

    classes = list(test.class_indices.keys())
    # print(classes)
    count_classes = len(classes)
    no_of_samples = len(predictions)

    error_list = []
    y_pred = []
    errors = 0

    for i, pred in enumerate(predictions):
        file = test.filenames[i]
        pred_label = np.argmax(pred)
        true_label = test.labels[i]
        # print("Prediction", pred_label, "TrueLabel", true_label)
        if pred_label != true_label:
            # print("Misclassification has Occured")
            errors = errors + 1
            filename = test.filenames[i]
            error_class = classes[pred_label]
            error_data = (filename, error_class)
            error_list.append((error_data))
        y_pred.append(pred_label)

    accuracy = (1 - (errors / no_of_samples))*100
    msg = f'Total error count : {errors}, in {no_of_samples} tests for an accuracy of {accuracy:6.2f}'
    print(msg)

    ytrue = np.array(y_true)
    ypred = np.array(y_pred)

    from sklearn.metrics import f1_score

    f1score = f1_score(ytrue, ypred, average='weighted') * 100
    print("F1 - Score :", f1score)

    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(ytrue, ypred)
    print("Confusion Matrix \n", confusion_matrix, "\n")

    from sklearn.metrics import classification_report
    import seaborn as sns
    classification_report = classification_report(y_true, y_pred, target_names=classes, digits=4)
    print("Classification Report \n", classification_report)

    import datetime

    date = datetime.datetime.now().strftime("%d-%m")
    time = datetime.datetime.now().strftime("%H.%M")

    # Confusion Matrix
    if count_classes <= 16:
        cm = confusion_matrix
        # plot the confusion matrix
        plt.figure(figsize=(8,8))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='crest', cbar=False)
        plt.xticks(np.arange(count_classes) + 1, classes, rotation=90)
        plt.yticks(np.arange(count_classes) + 1, classes, rotation=0)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")

        import os
        dir = os.listdir("./")
        if "ConfusionMatrix" not in dir:
            os.mkdir("ConfusionMatrix")
        else:
            print("ConfusionMatrix - Directory Already Exists")

        try:
            part = model.name.split("_")[0] + '_' + date + '_' + time
            path = f'./ConfusionMatrix/{part}'+'.jpeg'
            plt.savefig(path, bbox_inches='tight')
        except:
            print('***********')
            print('Error!!!')
            print('***********')

        plt.show()

    plt.close()

    return confusion_matrix, classification_report


#### sample_image is obtained by calling the get_sample_image function with i_s as an argument. The function returns a sample image, and [0] is used to access the first image from the returned batch of images.

plt.imshow is used to display the sample image.

plt.show() displays the image.

plt.close() is called to close the current figure and free up memory.

Finally, print(i_s) is used to print the value of i_s, which represents the image size.

In [ ]:
sample_image = get_sample_image(i_s)[0]
plt.imshow(sample_image[0], cmap="Greys")
plt.show()
plt.close()
print(i_s)


## Basic CNN Block

In [ ]:
train, val, test = get_data(i_s, batch_size, rotation_range, width_shift_range, height_shift_range, shear_range,
                            zoom_range, horizontal_flip, fill_mode)

In [ ]:
print_samples(train)

The provided code creates a convolutional neural network (CNN) model using TensorFlow and Keras. It defines the architecture of the model and compiles it with the specified loss function, optimizer, and metrics. Finally, it displays a summary of the model and a visual representation using the visualkeras library.

In [ ]:
import datetime

model_pfx = 'BasicCNN'

date = datetime.datetime.now().strftime('%d.%m.%Y')
time = datetime.datetime.now().strftime("%H..%M")
model_name = model_pfx + "_inputSize_" + str(i_s) + "_" + str(date) + "_" + str(time)

with tf.name_scope(model_name):
    model = Sequential(name=model_name)

with tf.name_scope("conv1"):
    model.add(Conv2D(16, (5,5), 1,
                     activation="relu",
                     input_shape=(i_s, i_s, 1)))

    model.add(tf.keras.layers.MaxPool2D(pool_size=3))
    model.add(tf.keras.layers.BatchNormalization())

with tf.name_scope("conv2"):
    model.add(Conv2D(32, (3,3), 1,
                     activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(pool_size=3))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.3))


with tf.name_scope("conv3"):
    model.add(Conv2D(64, (3, 3), 1,
                     activation="relu"))
    # model.add(tf.keras.layers.MaxPool2D(pool_size=3))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.3))

with tf.name_scope("conv4"):
    model.add(Conv2D(128, (3, 3), 1,
                     activation="relu"))
    model.add(tf.keras.layers.MaxPool2D(pool_size=3))
    model.add(tf.keras.layers.BatchNormalization())

with tf.name_scope("conv5"):
    model.add(Conv2D(256, (3, 3), 1,
                     activation="relu"))
    model.add(tf.keras.layers.BatchNormalization())

with tf.name_scope("flatten"):
    model.add(Flatten())
    model.add(tf.keras.layers.Dropout(0.4))

with tf.name_scope("dense"):
    # Adding Fully Connected Layers to the model
    model.add(Dense(64, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(16, activation='softmax'))

with tf.name_scope("compile"):
    model.compile(loss=loss, optimizer=opt, metrics=metric)

model.summary()

import visualkeras
visualkeras.layered_view(model, legend=True, scale_xy=1)


In [ ]:
basic_CNN, model_filepath = start_training(model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))

In [ ]:
cm_basicCNN, cr_basicCNN = confusion_matrix(model_filepath, test)

In [ ]:
model_filepath='Saved_Models/BasicCNN_inputSize_256_15.05.2023_20..56/BasicCNN_inputSize_256_15.05.2023_20..56_trained_on_87479_Data.h5'
extract_hidden_layer_data(model_filepath, sample_image, 8)

In [ ]:
del train, test, val, model_filepath

## Alexnet Architecture

In [ ]:
model = "AlexNet"
import datetime
date = datetime.datetime.now().strftime('%d.%m.%Y')
time = datetime.datetime.now().strftime("%H..%M")
model_name = model + "_inputSize_" + str(i_s) + "_" + str(date) + "_" + str(time)


In [ ]:
train, val, test = get_data(i_s, batch_size, rotation_range, width_shift_range, height_shift_range, shear_range,
                            zoom_range, horizontal_flip, fill_mode)

print_samples(train)

In [ ]:
from keras.layers import LeakyReLU

model = Sequential(name=model_name)

model.add(Conv2D(96, (11, 11), strides=4, input_shape=(i_s, i_s, 1)))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling2D(pool_size=3, strides=2))
model.add(BatchNormalization())

model.add(ZeroPadding2D(padding=2))
model.add(Conv2D(512, (5, 5), strides=1))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling2D(pool_size=3, strides=2))
model.add(BatchNormalization())

# model.add(ZeroPadding2D(padding=1))
# model.add(Conv2D(512, (3, 3), strides=1))
# model.add(LeakyReLU(alpha=0.2))

model.add(Flatten())
model.add(Dropout(0.4))

model.add(Dense(256))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.5))

model.add(Dense(16, activation="softmax"))

learning_rate = 0.001
opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)

model.compile(loss=loss, optimizer=opt, metrics=metric)

model.summary()


In [ ]:
AlexNet_Model, model_filepath = start_training(model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))

In [ ]:
model_filepath = 'Saved_Models/AlexNet_inputSize_256_16.05.2023_15..51/AlexNet_inputSize_256_16.05.2023_15..51_trained_on_87479_Data.h5'
AlexNet_Model, model_filepath = retrain_model(model_filepath, 0.0001, 30, train, val)

In [ ]:
cm_alexnet, cr_alexnet = confusion_matrix(model_filepath, test)

In [ ]:
extract_hidden_layer_data(model_filepath, sample_image, 32)

In [ ]:
       del train, val, test, model_filepath

### VGG-19 Architecture Block

In [ ]:
import datetime
# epochs = 30
# batch_size = 32
# i_s = 256 #Image_Size
# patience = 5

model = "VGG-19"
date = datetime.datetime.now().strftime('%d.%m.%Y')
time = datetime.datetime.now().strftime("%H..%M")
model_name = model + "_inputSize_" + str(i_s) + "_" + str(date) + "_" + str(time)

In [ ]:
train, val, test = get_data(i_s, batch_size, rotation_range, width_shift_range, height_shift_range, shear_range,
                            zoom_range, horizontal_flip, fill_mode)

In [ ]:
print_samples(train)

In [ ]:
model = Sequential(name=model_name)

model.add(Conv2D(64, (3, 3), 1,
                 activation="relu",
                 input_shape=(i_s, i_s, 1)))
model.add(Conv2D(64, (3, 3), 1,
                 activation="relu"))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(128, (3, 3), 1,
                 activation="relu"))
model.add(Conv2D(128, (3, 3), 1,
                 activation="relu"))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(256, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(Conv2D(256, (3, 3), 1,
                 activation="relu",
                 padding="same"))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(256, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(Conv2D(256, (3, 3), 1,
                 activation="relu",
                 padding="same"))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.BatchNormalization())

model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))
model.add(Conv2D(512, (3, 3), 1,
                 activation="relu",
                 padding="same"))

model.add(MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.BatchNormalization())

model.add(Flatten())

model.add(Dense(64, activation="relu"))
model.add(layers.Dropout(rate=0.5))

model.add(Dense(64, activation="relu"))

model.add(Dense(16, activation='softmax'))

model.compile(loss=loss, optimizer=opt, metrics=metric)

model.summary()

import visualkeras
visualkeras.layered_view(model, legend=True, scale_xy=0.6)

In [ ]:
VGG_19_Model, model_filepath = start_training(model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))

In [ ]:
loaded_model = tf.keras.models.load_model('Saved_Models/VGG-19_inputSize_256_09.05.2023_01..14/VGG-19_inputSize_256_09.05.2023_01..14_trained_on_87479_Data.h5')
learning_rate = 0.0001
previous_opt = loaded_model.optimizer
previous_opt.learning_rate.assign(learning_rate)
VGG_19_Model, model_filepath = start_training(loaded_model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))


In [ ]:
model_filepath='Saved_Models/VGG-19_inputSize_256_10.05.2023_11..24/VGG-19_inputSize_256_10.05.2023_11..24_trained_on_87479_Data.h5'

In [ ]:
cm_vgg_19, cr_vgg_19 = confusion_matrix(model_filepath, test)

In [ ]:
extract_hidden_layer_data(model_filepath, sample_image, 32)

In [ ]:
del train, val, test, model_filepath

VGG-16 Pretrained Model

In [ ]:
import datetime
# epochs = 30
# batch_size = 32
# i_s = 256 #Image_Size
# patience = 5

model = "VGG-16_Pretrained"
date = datetime.datetime.now().strftime('%d.%m.%Y')
time = datetime.datetime.now().strftime("%H..%M")
model_name = model + "_inputSize_" + str(i_s) + "_" + str(date) + "_" + str(time)

In [ ]:
train, val, test = get_data(i_s, batch_size, rotation_range, width_shift_range, height_shift_range, shear_range,
                            zoom_range, horizontal_flip, fill_mode, "rgb")

In [ ]:
# train.next()
print_samples(train)

In [ ]:
from tensorflow.keras.applications import VGG16
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(i_s, i_s, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers[:15]:
    layer.trainable = False

# Add our own layers on top of the pre-trained model
model = Sequential(name=model_name)
model.add(base_model)
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(16, activation='softmax'))
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=loss, optimizer=opt, metrics=monitor)

model.summary()

import visualkeras
visualkeras.layered_view(model, legend=True, scale_xy=0.6)

In [ ]:
VGG16_Pretrained_Model, model_filepath = start_training(model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))

In [ ]:
cm_vgg_16, cr_vgg_16 = confusion_matrix(model_filepath, test)

In [ ]:
extract_hidden_layer_data(model_filepath, sample_image, 32)

In [ ]:
del train, val, test, model_filepath

### ResNet Architecture

In [ ]:
import datetime

# epochs = 30
# batch_size = 32
# i_s = 256 #Image_Size
# patience = 5
model = "ResNet"

date = datetime.datetime.now().strftime('%d.%m.%Y')
time = datetime.datetime.now().strftime("%H..%M")
model_name = f'{model}_i_s_{i_s}_{date}_{time}'


In [ ]:
train, val, test = get_data(i_s, batch_size, rotation_range, width_shift_range, height_shift_range, shear_range,
                            zoom_range, horizontal_flip, fill_mode)

In [ ]:
print_samples(train)

In [ ]:
def identity_block(x, filters):
    x_input = x
    x = tf.keras.layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Add()([x, x_input])
    x = tf.keras.layers.Activation('relu')(x)
    return x


def convolutional_block(x, filters):
    x_input = x
    x = tf.keras.layers.Conv2D(filters, (3, 3), padding='same', strides=(2, 2))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x_input = tf.keras.layers.Conv2D(filters, (1, 1), strides=(2, 2))(x_input)
    x = tf.keras.layers.Add()([x, x_input])
    x = tf.keras.layers.Activation('relu')(x)
    return x


def ResNet34(shape=(i_s, i_s, 1), classes=16):
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    x = tf.keras.layers.Conv2D(16, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    # block_layers = [3, 4, 6, 3]
    block_layers = [2,4,6,2]

    filter_size = 16

    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            filter_size = filter_size * 2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)

    x = tf.keras.layers.AveragePooling2D((2, 2), padding='same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=x_input, outputs=x, name=model_name)
    return model


In [ ]:
model = ResNet34()

learning_rate = 0.01
opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)

model.compile(optimizer=opt,
              loss=loss,
              metrics=["accuracy"])
model.summary()

import visualkeras
visualkeras.layered_view(model, legend=True, scale_xy=1)

In [ ]:

Resnet_Model, model_filepath = start_training(model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))

In [ ]:
,cm_resnet, cr_resnet = confusion_matrix(model_filepath, test)

In [ ]:
extract_hidden_layer_data('Saved_Models/Pretrained_ResNet_50_i_s_24_12.05.2023_08..30/Pretrained_ResNet_50_i_s_24_12.05.2023_08..30_trained_on_87479_Data.h5', sample_image, 8)

In [ ]:
del train, test, val

## CHAT GPT RESNET

In [ ]:
import datetime

# epochs = 30
# batch_size = 32
# i_s = 256 #Image_Size
# patience = 5
model = "Pretrained_ResNet_50"

date = datetime.datetime.now().strftime('%d.%m.%Y')
time = datetime.datetime.now().strftime("%H..%M")
model_name = f'{model}_i_s_{i_s}_{date}_{time}'
i_s = 224

In [ ]:
train, val, test = get_data(i_s, batch_size, rotation_range, width_shift_range, height_shift_range, shear_range,
                            zoom_range, horizontal_flip, fill_mode, color='rgb')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Set the number of classes in the RVL-CDIP dataset
num_classes = 16

# Load the ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# base_model.summary()
# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)

# Add the final classification layer
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers[:-5]:
    layer.trainable = False

# Compile the model
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

import visualkeras
visualkeras.layered_view(model, legend=True)
# Train the model


In [ ]:

pretrained_Resnet_50_model, model_filepath = start_training(model, model_name, epochs, train, val, patience=patience, training_samples=str(train.samples))

In [ ]:
extract_hidden_layer_data('Saved_Models/VGG-19_inputSize_256_09.05.2023_01..14/VGG-19_inputSize_256_09.05.2023_01..14_trained_on_87479_Data.h5', sample_image, 16)

Model Comparison

In [ ]:
import os
import numpy as np

root = "./Models_Final"
directory = os.listdir(root)

sample_image = get_sample_image(i_s)[0]
plt.imshow(sample_image[0], cmap="Greys")
plt.show()
plt.close()
print(i_s)

train, val, test = get_data(i_s, batch_size, rotation_range, width_shift_range, height_shift_range, shear_range,
                            zoom_range, horizontal_flip, fill_mode)

for dir in directory:
    path_temp = os.path.join(root, dir)
    if(os.path.isdir(path_temp)):
        path_dir = os.listdir(path_temp)
        for entry in path_dir:
            path = os.path.join(path_temp, entry)
            if(path.endswith(".h5")):
                print(f'Working on -- {path}')


                model = tf.keras.models.load_model(path)

                model_summary_path = os.path.join(path_temp, f"{entry}_summary.jpeg")
                print(model_summary_path)
                tf.keras.utils.plot_model(model, to_file=model_summary_path, show_shapes=True)

                # model.summary()

                # confusion_matrix(path, test)
                print(f".... END .... {entry}")

            extract_hidden_layer_data(path, sample_image, 16)
